In [ ]:
!pip install -e /home/jovyan/Robbi/dea-notebooks/Tools/

In [ ]:
!pip install pyTMD==2.0.8

In [1]:
cd ../..

/home/jovyan/Robbi/dea-intertidal


In [2]:
%load_ext autoreload
%autoreload 2

import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from dea_tools.validation import eval_metrics
from intertidal.utils import round_date_strings

os.environ["DEA_TOOLS_TIDE_MODELS"] = "/home/jovyan/tide_models_clipped"

In [3]:
import glob
import warnings
import datetime
from odc.geo.geom import BoundingBox



def _load_gauge_metadata(metadata_path):
    
    # Load metadata
    metadata_df = pd.read_csv(metadata_path)
    metadata_df.columns = (
        metadata_df.columns.str.replace(" ", "_", regex=False)
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
        .str.replace("/", "_", regex=False)
        .str.lower()
    )
    metadata_df = metadata_df.set_index("site_code")

    # Convert metadata to GeoDataFrame
    metadata_gdf = gpd.GeoDataFrame(
        data=metadata_df,
        geometry=gpd.points_from_xy(metadata_df.longitude, metadata_df.latitude),
        crs="EPSG:4326",
    )
    
    return metadata_df, metadata_gdf


def tide_gauge_abslmp(
    x=None,
    y=None,
    site_code=None,
    time=("2020", "2021"),
    ahd=True,
    site_metadata=True,
    data_path="/gdata1/data/sea_level/abslmp/",
    metadata_path="/gdata1/data/sea_level/ABSLMP_station_metadata_v2.csv",
):
    """
    Load and process Australian Baseline Sea Level Monitoring Program
    (ABSLMP) tide gauge data.

    Parameters
    ----------
    x, y : tuple, optional
        Tuples defining the x and y bounding box within which to load
        tide gauge data, in WGS84 (degrees latitude, longitude) units.
        Leave as None if providing a list of site codes using 'site_code'.
    site_code : str or list of str, optional
        ABSLMP site code(s) for which to load data. If provided, 'x' and
        'y' will be ignored.
    time : tuple or list of str, optional
        Time range to consider, given as a tuple of start and end years.
        If None, will default to all tide observations from 1991 onward.
        Default is ("2020", "2021").
    ahd : bool, optional
        Whether to correct sea level to Australian Height Datum (AHD).
        Default is True.
    site_metadata : bool, optional
        Whether to add tide gauge station metadata as additional columns
        in the output DataFrame. Defaults to True.
    data_path : str, optional
        Path to the raw ABSLMP data files. Default is
        "/gdata1/data/sea_level/abslmp/".
    metadata_path : str, optional
        Path to the ABSLMP station metadata file.
        Default is "/gdata1/data/sea_level/ABSLMP_station_metadata_v2.csv".

    Returns
    -------
    pd.DataFrame
        Processed ABSLMP data as a DataFrame with columns including:
        "time": Timestamps,
        "sea_level": Observed sea level (m),
        "residuals": Residuals data (m),
        and additional columns from station metadata.
    """

    def _load_abslmp_dataset(path, na_value):
        abslmp_df = (
            pd.read_csv(
                path,
                parse_dates=[" Date & UTC Time"],
                na_values=na_value,
                usecols=[" Date & UTC Time", "Sea Level", "Residuals"],
            )
            .rename(
                {
                    " Date & UTC Time": "time",
                    "Sea Level": "sea_level",
                    "Residuals": "residuals",
                },
                axis=1,
            )
            .assign(site_code=path[-17:-9])
            .set_index("time")
        )

        return abslmp_df

    # Load tide gauge metadata
    metadata_df, metadata_gdf = _load_gauge_metadata(metadata_path)

    # Use supplied site codes if available
    if site_code is not None:
        site_code = [site_code] if isinstance(site_code, str) else site_code

    # Otherwise, use xy bounds to identify sites
    elif x is not None:
        bbox = BoundingBox.from_xy(x, y)
        site_code = metadata_gdf.cx[
            bbox.left : bbox.right, bbox.top : bbox.bottom
        ].index

    # Otherwise, return all available site codes
    else:
        site_code = metadata_df.index.to_list()

    # Prepare times
    if time is None:
        time = ["1991", str(datetime.datetime.now().year)]
    time = [time] if isinstance(time, str) else time
    start_time = round_date_strings(time[0], round_type="start")
    end_time = round_date_strings(time[-1], round_type="end")

    # Identify paths to load and nodata values for each site
    years = list(range(int(start_time[0:4]), int(end_time[0:4]) + 1))
    paths_na = [
        (glob.glob(f"{data_path}/{s}_*{y}.csv"), metadata_df.loc[s].null_value)
        for y in years
        for s in site_code
    ]

    # Expand so we have a nodata value for each path, then load and
    # combine into a single dataframe
    paths_na = [(path, na) for paths, na in paths_na for path in paths]
    data_df = (
        pd.concat([_load_abslmp_dataset(path, na_value=na) for path, na in paths_na])
        .loc[slice(start_time, end_time)]
        .reset_index()
        .set_index("site_code")
    )

    # Insert metadata into dataframe
    data_df[metadata_df.columns] = metadata_df

    # Add time to index and remove duplicates
    data_df = data_df.set_index("time", append=True)
    duplicates = data_df.index.duplicated()
    if duplicates.sum() > 0:
        warnings.warn("Duplicate timestamps were removed.")
        data_df = data_df.loc[~duplicates]

    # Correct to AHD (i.e. mean sea level)
    if ahd:
        data_df["sea_level"] -= data_df.ahd

    # Return data
    if not site_metadata:
        return data_df[["sea_level", "residuals"]]
    else:
        return data_df


def tide_gauge_gesla(
    x=None,
    y=None,
    site_code=None,
    time=("2020", "2021"),
    filter_use_flag=True,
    site_metadata=True,
    data_path="/gdata1/data/sea_level/gesla/",
    metadata_path="/gdata1/data/sea_level/GESLA3_ALL 2.csv",
):
    """
    Load and process Global Extreme Sea Level Analysis (GESLA) tide
    gauge data.

    Modified from original code from https://github.com/philiprt/GeslaDataset.

    Parameters
    ----------
    x, y : tuple, optional
        Tuples defining the x and y bounding box within which to load
        tide gauge data, in WGS84 (degrees latitude, longitude) units.
        Leave as None if providing a list of site codes using 'site_code'.
    site_code : str or list of str, optional
        GESLA site code(s) for which to load data. If provided, 'x' and
        'y' will be ignored.
    time : tuple or list of str, optional
        Time range to consider, given as a tuple of start and end years.
        If None, will default to all tide observations from 1800 onward.
        Default is ("2020", "2021").
    filter_use_flag : bool, optional
        Whether to filter out low quality observations with a "use_flag"
        value of 0 (do not use). Defaults to True.
    site_metadata : bool, optional
        Whether to add tide gauge station metadata as additional columns
        in the output DataFrame. Defaults to True.
    data_path : str, optional
        Path to the raw GESLA data files. Default is
        "/gdata1/data/sea_level/gesla/".
    metadata_path : str, optional
        Path to the GESLA station metadata file.
        Default is "/gdata1/data/sea_level/GESLA3_ALL 2.csv".

    Returns
    -------
    pd.DataFrame
        Processed GESLA data as a DataFrame with columns including:
        "time": Timestamps,
        "sea_level": Observed sea level (m),
        "qc_flag": Observed sea level QC flag,
        "use_flag": Use-in-analysis flag (1 = use, 0 = do not use),
        and additional columns from station metadata.
    """

    def _load_gesla_dataset(site, path, na_value):
        gesla_df = (
            pd.read_csv(
                path,
                skiprows=41,
                names=["date", "time", "sea_level", "qc_flag", "use_flag"],
                sep="\s+",
                parse_dates=[[0, 1]],
                index_col=0,
                na_values=na_value,
            )
            .rename_axis("time")
            .assign(site_code=site)
        )

        return gesla_df

    
    # Load tide gauge metadata
    metadata_df, metadata_gdf = _load_gauge_metadata(metadata_path)    

    # Use supplied site codes if available
    if site_code is not None:
        site_code = [site_code] if isinstance(site_code, str) else site_code

    # Otherwise, use xy bounds to identify sites
    elif x is not None:
        bbox = BoundingBox.from_xy(x, y)
        site_code = metadata_gdf.cx[
            bbox.left : bbox.right, bbox.top : bbox.bottom
        ].index

    # Otherwise, return all available site codes
    else:
        site_code = metadata_df.index.to_list()

    # Prepare times
    if time is None:
        time = ["1800", str(datetime.datetime.now().year)]
    time = [time] if isinstance(time, str) else time
    start_time = round_date_strings(time[0], round_type="start")
    end_time = round_date_strings(time[-1], round_type="end")

    # Identify paths to load and nodata values for each site
    metadata_df["file_name"] = data_path + metadata_df["file_name"]
    paths_na = metadata_df.loc[site_code, ["file_name", "null_value"]]

    # Load and combine into a single dataframe
    data_df = (
        pd.concat(
            [
                _load_gesla_dataset(s, p, na_value=na)
                for s, p, na in paths_na.itertuples()
            ]
        )
        .sort_index()
        .loc[slice(start_time, end_time)]
        .reset_index()
        .set_index("site_code")
    )

    # Optionally filter by use flag column
    if filter_use_flag:
        data_df = data_df.loc[data_df.use_flag == 1]

    # Optionally insert metadata into dataframe
    if site_metadata:
        data_df[metadata_df.columns] = metadata_df.loc[site_code]

    # Add time to index and remove duplicates
    data_df = data_df.set_index("time", append=True)
    duplicates = data_df.index.duplicated()
    if duplicates.sum() > 0:
        warnings.warn("Duplicate timestamps were removed.")
        data_df = data_df.loc[~duplicates]

    # Return data
    return data_df


# tide_gauge_abslmp(x=(140, 160), y=(-30, -35))
# tide_gauge_gesla(x=(140, 160), y=(-30, -35))

### GESLA tide guage data


In [4]:
# # Load tide gauge metadata
# metadata_df, metadata_gdf = _load_gauge_metadata(metadata_path="/gdata1/data/sea_level/GESLA3_ALL 2.csv") 
# metadata_gdf.to_file("gesla_stations.geojson")

In [5]:
# Load Collection 3 summary grid
c3_path = "https://data.dea.ga.gov.au/derivative/ga_summary_grid_c3.geojson"
c3_grid = gpd.read_file(c3_path)
xmin, ymin, xmax, ymax = c3_grid.total_bounds

In [6]:
# Extract tide data for all sites
gauge_df = tide_gauge_gesla(x=(xmin, xmax), y=(ymax, ymin), time=("2017", "2019"))

In [7]:
# Clean to restrict to Australia and remove duplicate sites/contributers
bad_sites = ("H033007A",  # Cape Ferguson, duplicate
             "H057022B",  # Thursday Island, duplicate
             "PLPEE01",  # Peel Inlet, inland
             "DVHAR01", # Harvey, inland
             "H060010A") # Half Tide Tug, duplicate 
bad_contributers = ("UHSLC",)
gauge_df = gauge_df.query(
    f"(country == 'AUS') & "
    f"(site_code not in {bad_sites}) & "
    f"(contributor_abbreviated not in {bad_contributers})"
)

# Normalise to mean sea level
gauge_df["sea_level"] -= gauge_df.groupby(["site_code"])["sea_level"].transform("mean")

# Select 3-hourly subset
# gauge_df = gauge_df.iloc[::3]

In [ ]:
# gauge_df.query('time < 20200101').groupby(gauge_df.query('time < 20200101').index.get_level_values("time").year).sea_level.std().plot()

In [ ]:
sites_df = gauge_df.groupby("site_code").first().iloc[:, 3:]
sites_gdf = gpd.GeoDataFrame(data=sites_df, geometry=gpd.points_from_xy(sites_df.longitude, sites_df.latitude))
sites_gdf.to_file("gesla_stations_aus.geojson")

In [8]:
from dea_tools.coastal import model_tides

tide_df = model_tides(
    x=gauge_df.longitude,
    y=gauge_df.latitude,
    time=gauge_df.index.get_level_values("time"),
    model=[
        "FES2014",
        "FES2012",
        "TPXO8-atlas-v1",
        "TPXO9-atlas-v5",
        "EOT20",
        "HAMTIDE11",
        "GOT4.10",
    ],
    mode="one-to-one",
    parallel_splits=100,
    output_format="wide",
).rename(
    {
        "TPXO9-atlas-v5": "TPXO9",
        "TPXO8-atlas": "TPXO8",
        "TPXO8-atlas-v1": "TPXO8",
    },
    axis=1,
)
tide_df

Modelling tides using FES2014, FES2012, TPXO8-atlas-v1, TPXO9-atlas-v5, EOT20, HAMTIDE11, GOT4.10 in parallel


100%|██████████| 700/700 [05:57<00:00,  1.96it/s]


Converting to a wide format dataframe


tide_model                           EOT20   FES2012   FES2014   GOT4.10  \
time       x          y                                                    
2017-01-01 152.508211 -32.173989  0.784791  0.850682  0.825635  0.799521   
           147.973100 -37.885200  0.545370  0.561856  0.587374  0.545110   
           150.476500 -35.357700  0.726228  0.761144  0.737824  0.717146   
           146.833300 -19.250000  0.971019  1.029977  1.020217  1.012640   
           147.341000 -42.877300  0.371824  0.435218  0.405258  0.403980   
...                                    ...       ...       ...       ...   
2019-12-31 150.759200 -34.906600  0.470200  0.502368  0.490144  0.475990   
           151.220400 -33.968400  0.448499  0.515940  0.504787  0.490595   
           121.895400 -33.870900 -0.256765 -0.189922 -0.202952 -0.193056   
           115.629983 -32.601376 -0.261014 -0.195017 -0.206508 -0.219800   
           135.870100 -34.715900 -0.402618 -0.342923 -0.321599 -0.230691   

tide_model                        HAMTIDE11     TPXO8     TPXO9  
time       x          y                                          
2017-01-01 152.508211 -32.173989   0.837228  0.835900  0.817941  
           147.973100 -37.885200   0.539387  0.506428  0.469878  
           150.476500 -35.357700   0.741500  0.723237  0.722399  
           146.833300 -19.250000   1.075376  0.919476  1.036068  
           147.341000 -42.877300   0.397241  0.375177  0.392107  
...                                     ...       ...       ...  
2019-12-31 150.759200 -34.906600   0.499208  0.481597  0.492294  
           151.220400 -33.968400   0.512284  0.476872  0.507859  
           121.895400 -33.870900  -0.166687 -0.181433 -0.178121  
           115.629983 -32.601376  -0.195348 -0.176052 -0.029747  
           135.870100 -34.715900  -0.297189 -0.254946 -0.258349  

[5337216 rows x 7 columns]

In [9]:
# Add tide gauge data to dataframe
tide_df["site_code"] = gauge_df.index.get_level_values("site_code").values
tide_df["site_name"] = gauge_df.site_name.values
tide_df["tide_gauge"] = gauge_df.sea_level.values

# Reshape to long format
tide_df_long = tide_df.melt(
    ignore_index=False,
    id_vars=["tide_gauge", "site_code", "site_name"],
    value_vars=[
        "EOT20",
        "FES2012",
        "FES2014",
        "GOT4.10",
        "HAMTIDE11",
        "TPXO9",
        "TPXO8",
    ],
    value_name="tide_m",
)

tide_df_long

tide_gauge site_code  \
time       x          y                                  
2017-01-01 152.508211 -32.173989    0.702968    209402   
           147.973100 -37.885200    0.423943     6057a   
           150.476500 -35.357700    0.730486    216471   
           146.833300 -19.250000    0.980192     59250   
           147.341000 -42.877300    0.436934     61220   
...                                      ...       ...   
2019-12-31 150.759200 -34.906600    0.354471    215408   
           151.220400 -33.968400    0.410569     60390   
           121.895400 -33.870900   -0.243620     62080   
           115.629983 -32.601376   -0.332215   DVBVD01   
           135.870100 -34.715900   -0.289914     61900   

                                                            site_name  \
time       x          y                                                 
2017-01-01 152.508211 -32.173989                              Forster   
           147.973100 -37.885200  Lakes_Entrance_Inner_Bullock_Island   
           150.476500 -35.357700                            Ulladulla   
           146.833300 -19.250000                           Townsville   
           147.341000 -42.877300                               Hobart   
...                                                               ...   
2019-12-31 150.759200 -34.906600                     Crookhaven_Heads   
           151.220400 -33.968400                           Botany_Bay   
           121.895400 -33.870900                            Esperance   
           115.629983 -32.601376                         Cape_Bouvard   
           135.870100 -34.715900                         Port_Lincoln   

                                 tide_model    tide_m  
time       x          y                                
2017-01-01 152.508211 -32.173989      EOT20  0.784791  
           147.973100 -37.885200      EOT20  0.545370  
           150.476500 -35.357700      EOT20  0.726228  
           146.833300 -19.250000      EOT20  0.971019  
           147.341000 -42.877300      EOT20  0.371824  
...                                     ...       ...  
2019-12-31 150.759200 -34.906600      TPXO8  0.481597  
           151.220400 -33.968400      TPXO8  0.476872  
           121.895400 -33.870900      TPXO8 -0.181433  
           115.629983 -32.601376      TPXO8 -0.176052  
           135.870100 -34.715900      TPXO8 -0.254946  

[37360512 rows x 5 columns]

## Analysis
### Export tide validation plots

In [ ]:
# Calculate tide range per site
tide_range = (
    tide_df_long.groupby("site_code")["tide_gauge"]
    .apply(lambda x: np.abs(x).max())
    .sort_values()
)

# Select subset
# sites, limits, title = tide_range.loc[tide_range >= 2].index, 5, "Macrotidal sites"
# sites, limits, title = tide_range.loc[(tide_range >= 1) & (tide_range < 2)].index, 2, "Mesotidal sites"
# sites, limits, title = tide_range.loc[tide_range < 1].index, 1, "Microtidal sites"

# Optionally restrict to subset
site_filter = [
    "DYDBY01",
    "63090",
    "59511",
    "59510",
    "61800",
    "59690",
    "61840",
    "61600",
    "58170",
    "60780",
    "60739",
    "60590",
    "63511",
    "59980",
    "60710",
    "60730",
]
sites, limits, title = site_filter, 5, "Problematic sites" 

# Plot facetted
g = sns.FacetGrid(
    tide_df_long.query(f"site_code in {sites}").reset_index(),
    col="tide_model",
    row="site_name",
    margin_titles=True,
    xlim=(-limits, limits),
    ylim=(-limits, limits),
)
g.fig.suptitle(title, size=20)
g.set_titles(row_template="{row_name}", col_template="{col_name}")
g.map(sns.scatterplot, "tide_gauge", "tide_m", alpha=0.3, linewidth=0, s=3)
for a in g.axes.flat:
    a.plot([-limits, limits], [-limits, limits], "--", c="black")
g.savefig(f"{title.replace(' ', '')}.jpg")

### Calculate accuracy stats for each site and model

In [11]:
accuracy_df = tide_df_long.groupby(["tide_model"])[
    ["tide_gauge", "tide_m"]
].apply(lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m, round=4))
accuracy_df

,Correlation,RMSE,MAE,R-squared,Bias,Regression slope
tide_model,,,,,,
EOT20,0.9347,0.1976,0.1325,0.8737,0.0003,0.8759
FES2012,0.9108,0.2313,0.1504,0.8269,0.0061,0.8756
FES2014,0.9253,0.2109,0.1429,0.8562,-0.0001,0.8632
GOT4.10,0.8919,0.2515,0.1586,0.7954,-0.0001,0.8040
HAMTIDE11,0.9103,0.2302,0.1517,0.8285,-0.0001,0.8204
TPXO8,0.9050,0.2383,0.1450,0.8162,-0.0001,0.8675
TPXO9,0.9317,0.2020,0.1380,0.8679,0.0032,0.8730


### Best model per site

In [12]:
accuracy_sites_df = tide_df_long.groupby(["site_name", "tide_model", "x", "y"])[
    ["tide_gauge", "tide_m"]
].apply(lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m, round=4))
accuracy_sites_df

Correlation    RMSE     MAE  \
site_name tide_model x        y                                       
Albany    EOT20      117.8926 -35.0337       0.8419  0.1229  0.0954   
          FES2012    117.8926 -35.0337       0.8109  0.1328  0.1044   
          FES2014    117.8926 -35.0337       0.8130  0.1320  0.1033   
          GOT4.10    117.8926 -35.0337       0.8122  0.1323  0.1035   
          HAMTIDE11  117.8926 -35.0337       0.8128  0.1322  0.1034   
...                                             ...     ...     ...   
Yamba     FES2014    153.3621 -29.4290       0.9630  0.1213  0.0960   
          GOT4.10    153.3621 -29.4290       0.9615  0.1184  0.0931   
          HAMTIDE11  153.3621 -29.4290       0.9634  0.1180  0.0928   
          TPXO8      153.3621 -29.4290       0.9632  0.1186  0.0935   
          TPXO9      153.3621 -29.4290       0.9633  0.1156  0.0910   

                                        R-squared    Bias  Regression slope  
site_name tide_model x        y                                              
Albany    EOT20      117.8926 -35.0337     0.7060  0.0001            0.6652  
          FES2012    117.8926 -35.0337     0.6568  0.0061            0.6481  
          FES2014    117.8926 -35.0337     0.6606  0.0000            0.6447  
          GOT4.10    117.8926 -35.0337     0.6593  0.0000            0.6461  
          HAMTIDE11  117.8926 -35.0337     0.6599  0.0000            0.6376  
...                                           ...     ...               ...  
Yamba     FES2014    153.3621 -29.4290     0.9044  0.0000            1.0735  
          GOT4.10    153.3621 -29.4290     0.9090 -0.0000            1.0438  
          HAMTIDE11  153.3621 -29.4290     0.9095 -0.0000            1.0596  
          TPXO8      153.3621 -29.4290     0.9086 -0.0000            1.0610  
          TPXO9      153.3621 -29.4290     0.9132  0.0000            1.0449  

[665 rows x 6 columns]

In [13]:
accuracy_sites_df.loc[
    accuracy_sites_df.groupby("site_name").RMSE.idxmin()
].reset_index().tide_model.value_counts()

EOT20        63
TPXO9        14
FES2014       5
TPXO8         4
FES2012       4
GOT4.10       3
HAMTIDE11     2
Name: tide_model, dtype: int64

### Tide gauge weighting

In [ ]:
models = ["EOT20", "FES2012", "FES2014", "GOT4.10", "HAMTIDE11", "TPXO8", "TPXO9"]

# Convert to wide
wide_df = accuracy_sites_df["RMSE"].unstack("tide_model").reset_index()
wide_df = wide_df.set_index("site_name")[models]
wide_df

#### Correlation weighting (alternative)

In [21]:
models = ["EOT20", "FES2012", "FES2014", "GOT4.10", "HAMTIDE11", "TPXO8", "TPXO9"]
corr_results_gesla = gpd.read_file("corr_results_gesla_v3.geojson")

wide_df = (
    corr_results_gesla
    .rename(
        {"point_id": "site_name", "TPXO8-atlas-v1": "TPXO8", "TPXO9-atlas-v5": "TPXO9"},
        axis=1,
    )
    .set_index("site_name")[models]
    .dropna(axis=0)
)

wide_df = 1 - wide_df

wide_df

,EOT20,FES2012,FES2014,GOT4.10,HAMTIDE11,TPXO8,TPXO9
site_name,,,,,,,
Albany,0.695545,0.700939,0.704296,0.706396,0.705631,0.704950,0.702340
Batemans_Bay_Princess_Jetty,0.722653,0.727599,0.723360,0.724890,0.726719,0.723507,0.723951
Bermagui,0.687334,0.690656,0.687476,0.687516,0.690776,0.687995,0.688410
Booby_Island,0.761203,0.753744,0.756029,0.751888,0.762019,0.754929,0.757416
Botany_Bay,0.616258,0.625706,0.622091,0.621310,0.623512,0.624922,0.622817
...,...,...,...,...,...,...,...
Weipa_Humbug_Point,0.740850,0.728162,0.731809,0.722743,0.734452,0.731994,0.718759
Western_Port_Stony_Point,0.721061,0.693420,0.728076,0.696129,0.693915,0.687923,0.675731
Whyalla,0.824089,0.859067,0.814377,0.871608,0.798149,0.828470,0.813053


### Weight application

In [22]:
def weighted_mean_top_n(gauge_df, altimetry_df, top_n=5):
    
    # Calculate ranks for each model
    ranks_df = altimetry_df.rank(axis=1, ascending=False, method="max")
    
    # Select top N models
    altimetry_weights = (ranks_df - (len(wide_df.columns) - top_n)).clip(0, top_n) ** 10
 
    # Normalise to sum to 1
    altimetry_weights = altimetry_weights.divide(altimetry_weights.sum(axis=1), axis=0)    

    return (gauge_df * altimetry_weights).sum(axis=1)
    

In [23]:
# Prepare data
test_df = tide_df.set_index("site_name", append=True).copy()
test_df["site_code"] = gauge_df.index.get_level_values("site_code").values
test_df["tide_gauge"] = gauge_df.sea_level.values

# Apply different weighting schemes
test_df["median"] = test_df[models].median(axis=1)
# test_df["median_top5"] = median_top_n(test_df[models], wide_df, top_n=5)
test_df["median_top3"] = median_top_n(test_df[models], wide_df, top_n=3)
test_df["mean"] = test_df[models].mean(axis=1)
# test_df["mean_top5"] = mean_top_n(test_df[models], wide_df, top_n=5)
test_df["mean_top3"] = mean_top_n(test_df[models], wide_df, top_n=3)

# test_df["mean_top2"] = mean_top_n(test_df[models], wide_df, top_n=2)
# test_df["top_model"] = mean_top_n(test_df[models], wide_df, top_n=1)
# test_df["weighted_mean_power2"] = weighted_mean_power(test_df[models], wide_df, power=2)
# test_df["weighted_mean_power5"] = weighted_mean_power(test_df[models], wide_df, power=5)
# test_df["weighted_mean_power10"] = weighted_mean_power(test_df[models], wide_df, power=10)
# test_df["weighted_mean_top3"] = weighted_mean_top_n(test_df[models], wide_df, top_n=3)
# test_df["weighted_mean_top5"] = weighted_mean_top_n(test_df[models], wide_df, top_n=5)

# Reshape to long format
tide_df_long = test_df.melt(
    ignore_index=False,
    id_vars=["tide_gauge", "site_code"],
    value_name="tide_m",
)

# Calculate statistics against tide gauge data
accuracy_df = tide_df_long.groupby(["tide_model"])[["tide_gauge", "tide_m"]].apply(
    lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m)
)
accuracy_df.sort_values("RMSE").style.background_gradient(
    cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5
)

,Correlation,RMSE,MAE,R-squared,Bias,Regression slope
tide_model,,,,,,
mean_top3,0.942000,0.187000,0.128000,0.888000,0.002000,0.884000
median_top3,0.940000,0.190000,0.130000,0.884000,0.001000,0.884000
median,0.937000,0.194000,0.134000,0.878000,0.001000,0.865000
mean,0.935000,0.197000,0.136000,0.874000,0.001000,0.854000
EOT20,0.935000,0.198000,0.133000,0.874000,0.000000,0.876000
TPXO9,0.932000,0.202000,0.138000,0.868000,0.003000,0.873000
FES2014,0.925000,0.211000,0.143000,0.856000,-0.000000,0.863000
HAMTIDE11,0.910000,0.230000,0.152000,0.828000,-0.000000,0.820000
FES2012,0.911000,0.231000,0.150000,0.827000,0.006000,0.876000


In [19]:
# Prepare data
test_df = tide_df.set_index("site_name", append=True).copy()
test_df["site_code"] = gauge_df.index.get_level_values("site_code").values
test_df["tide_gauge"] = gauge_df.sea_level.values

# Apply different weighting schemes
test_df["median"] = test_df[models].median(axis=1)
# test_df["median_top5"] = median_top_n(test_df[models], wide_df, top_n=5)
test_df["median_top3"] = median_top_n(test_df[models], wide_df, top_n=3)
test_df["mean"] = test_df[models].mean(axis=1)
# test_df["mean_top5"] = mean_top_n(test_df[models], wide_df, top_n=5)
test_df["mean_top3"] = mean_top_n(test_df[models], wide_df, top_n=3)

# test_df["mean_top2"] = mean_top_n(test_df[models], wide_df, top_n=2)
# test_df["top_model"] = mean_top_n(test_df[models], wide_df, top_n=1)
# test_df["weighted_mean_power2"] = weighted_mean_power(test_df[models], wide_df, power=2)
# test_df["weighted_mean_power5"] = weighted_mean_power(test_df[models], wide_df, power=5)
# test_df["weighted_mean_power10"] = weighted_mean_power(test_df[models], wide_df, power=10)
# test_df["weighted_mean_top3"] = weighted_mean_top_n(test_df[models], wide_df, top_n=3)
# test_df["weighted_mean_top5"] = weighted_mean_top_n(test_df[models], wide_df, top_n=5)

# Reshape to long format
tide_df_long = test_df.melt(
    ignore_index=False,
    id_vars=["tide_gauge", "site_code"],
    value_name="tide_m",
)

# Calculate statistics against tide gauge data
accuracy_df = tide_df_long.groupby(["tide_model"])[["tide_gauge", "tide_m"]].apply(
    lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m)
)
accuracy_df.sort_values("RMSE").style.background_gradient(
    cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5
)

,Correlation,RMSE,MAE,R-squared,Bias,Regression slope
tide_model,,,,,,
mean_top3,0.944000,0.185000,0.128000,0.891000,0.001000,0.882000
median_top3,0.943000,0.186000,0.130000,0.889000,0.001000,0.881000
median,0.937000,0.194000,0.134000,0.878000,0.001000,0.865000
mean,0.935000,0.197000,0.136000,0.874000,0.001000,0.854000
EOT20,0.935000,0.198000,0.133000,0.874000,0.000000,0.876000
TPXO9,0.932000,0.202000,0.138000,0.868000,0.003000,0.873000
FES2014,0.925000,0.211000,0.143000,0.856000,-0.000000,0.863000
HAMTIDE11,0.910000,0.230000,0.152000,0.828000,-0.000000,0.820000
FES2012,0.911000,0.231000,0.150000,0.827000,0.006000,0.876000


In [24]:
# Compare performance at worst sites
accuracy_sites_df = tide_df_long.dropna(axis=0).groupby(["tide_model", "site_code"])[
    ["tide_gauge", "tide_m"]
].apply(lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m))

accuracy_sites_df.groupby(["tide_model"]).quantile(0.9).sort_values(
    "RMSE"
).style.background_gradient(cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5)


,Correlation,RMSE,MAE,R-squared,Bias,Regression slope
tide_model,,,,,,
median_top3,0.989000,0.267800,0.217600,0.976000,0.006000,0.996800
mean_top3,0.989800,0.273400,0.220200,0.976200,0.004800,1.009800
TPXO9,0.989000,0.299800,0.246600,0.977000,0.011000,1.008800
median,0.988600,0.302600,0.246600,0.974600,0.004000,0.978600
TPXO8,0.983800,0.327400,0.269200,0.962200,0.000000,0.996600
EOT20,0.987200,0.327600,0.276000,0.973800,0.001000,0.999200
mean,0.984000,0.330600,0.275200,0.967000,0.003000,0.975800
FES2014,0.983000,0.340000,0.285400,0.966000,0.000000,0.991600
FES2012,0.985800,0.367800,0.295400,0.967800,0.010000,0.995600


In [20]:
# Compare performance at worst sites
accuracy_sites_df = tide_df_long.dropna(axis=0).groupby(["tide_model", "site_code"])[
    ["tide_gauge", "tide_m"]
].apply(lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m))

accuracy_sites_df.groupby(["tide_model"]).quantile(0.9).sort_values(
    "RMSE"
).style.background_gradient(cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5)



,Correlation,RMSE,MAE,R-squared,Bias,Regression slope
tide_model,,,,,,
median_top3,0.988800,0.259600,0.211400,0.976000,0.006000,0.997400
mean_top3,0.989800,0.277800,0.223200,0.975800,0.004000,1.010400
TPXO9,0.989000,0.299800,0.246600,0.977000,0.011000,1.008800
median,0.988600,0.302600,0.246600,0.974600,0.004000,0.978600
TPXO8,0.983800,0.327400,0.269200,0.962200,0.000000,0.996600
EOT20,0.987200,0.327600,0.276000,0.973800,0.001000,0.999200
mean,0.984000,0.330600,0.275200,0.967000,0.003000,0.975800
FES2014,0.983000,0.340000,0.285400,0.966000,0.000000,0.991600
FES2012,0.985800,0.367800,0.295400,0.967800,0.010000,0.995600


In [ ]:
accuracy_sites_df.query('site_code == "60780"').sort_values(
    "RMSE"
).style.background_gradient(cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5)

## Altimetry comparisons

In [ ]:
# Convert to GeoDataFrame
wide_df = accuracy_df["RMSE"].unstack("tide_model").reset_index()
wide_gdf = gpd.GeoDataFrame(
    data=wide_df, geometry=gpd.points_from_xy(x=wide_df.x, y=wide_df.y), crs="EPSG:4326"
)

In [ ]:
# Load in altimetry data
xtrack_rms_gdf = gpd.read_file("/home/jovyan/altimetry/X-TRACK/xtrack_rms_all.geojson")
xtrack_rms_gdf

In [ ]:
# gpd.sjoin_nearest(
#         gpd.GeoDataFrame(geometry=gpd.points_from_xy(x=[149.88344], y=[-22.33805], crs="EPSG:4326")),
#         xtrack_rms_gdf,
#         lsuffix="gauge",
#         rsuffix="",
#         how="left",
#         distance_col="dist",
#     )

In [ ]:
model_names = ["EOT20", "FES2012", "FES2014", "GOT4.10", "HAMTIDE11", "TPXO8", "TPXO9"]

wide_df = wide_gdf.set_index("site_name")[model_names]
xtrack_df = (
    gpd.sjoin_nearest(
        wide_gdf[["site_name", "geometry"]],
        xtrack_rms_gdf,
        lsuffix="gauge",
        rsuffix="",
        how="left",
        distance_col="dist",
    )
    .set_index("site_name")
)

# Keep distances between gauge and altimetry points
distances = xtrack_df.dist
xtrack_df = xtrack_df[model_names] 

### Difference between altimetry and gauge RMS

In [ ]:
with pd.option_context('display.max_rows', 500):
    display((xtrack_df - wide_df).reset_index().style.background_gradient(cmap='RdBu', axis=None, vmin=-0.3, vmax=0.3))

### RMS scatterplot

In [ ]:
gauge_df_long = wide_df.assign(distance=distances).melt(
    id_vars=["distance"], value_name="Tide gauge (metres RMS)", ignore_index=False
)
gauge_df_long["Altimetry (metres RMS)"] = xtrack_df.melt(ignore_index=False).value

fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(
    ax=ax,
    data=gauge_df_long,
    x="Tide gauge (metres RMS)",
    y="Altimetry (metres RMS)",
    hue="site_name",
    s=50,
    palette=sns.color_palette("tab20"),
    edgecolor="black",
    style="tide_model",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
ax.plot([0.05, 2], [0.05, 2], "--", c="black")
ax.set_title(
    "Comparison of tide model performance at tide\ngauge and nearest altimetry observation"
);

In [ ]:
import matplotlib 

fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(
    ax=ax,
    data=gauge_df_long,
    x="distance",
    y="Tide gauge (metres RMS)",
    hue="distance",
    # hue_norm=matplotlib.colors.LogNorm(),
    s=50,
    palette="magma",
    edgecolor="black",
    style="tide_model",
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
# ax.plot([0.05, 2], [0.05, 2], "--", c="black")
ax.set_title(
    "Comparison of tide model performance at tide\ngauge and nearest altimetry observation"
);

### Overall stats

In [ ]:
eval_metrics(x=gauge_df_long["Tide gauge (metres RMS)"], 
             y=gauge_df_long["Altimetry (metres RMS)"])

### Agreement between top/bottom models

In [ ]:
gauge_worst_n = gauge_df_long.groupby("site_name").apply(lambda x: x.nlargest(4, columns="Tide gauge (metres RMS)", keep='first')).set_index("tide_model", append=True)
xtrack_worst_n = gauge_df_long.groupby("site_name").apply(lambda x: x.nlargest(4, columns="Altimetry (metres RMS)", keep="first")).set_index("tide_model", append=True)

In [ ]:
gauge_worst_n[["Tide gauge (metres RMS)"]].join(xtrack_worst_n[["Altimetry (metres RMS)"]], rsuffix="_altimetry").groupby("site_name").count()

### Spearman's correlation between gauge and altimetry per site
High values indicate both data sources sort models into the same RMS order

In [ ]:
xtrack_df.corrwith(wide_df, axis=1, method="spearman").to_frame().style.background_gradient(cmap='RdBu', axis=None, vmin=-1, vmax=1)

## Weighting test

In [16]:
def weighted_median(values, weights):
    values = np.array(values)
    weights = np.array(weights)
    
    sort_indices = np.argsort(values)
    values_sorted = values[sort_indices]
    weights_sorted = weights[sort_indices]  

    cumsum = weights_sorted.cumsum()
    cutoff = weights_sorted.sum() / 2.
    return values_sorted[cumsum >= cutoff][0]


# def weighted_quantiles(values, weights, quantiles=0.5):
#     i = np.argsort(values)
#     c = np.cumsum(weights[i])
#     return values[i[np.searchsorted(c, np.array(quantiles) * c[-1])]]

# def weighted_median(df, val, weight):
#     df_sorted = df.sort_values(val)
#     cumsum = df_sorted[weight].cumsum()
#     cutoff = df_sorted[weight].sum() / 2.
#     return df_sorted[cumsum >= cutoff][val].iloc[0]

In [17]:
# weighted_median = test_df[model_names].apply(lambda x: weighted_median(x, xtrack_weights.loc[x.name[3]]), axis=1)

In [18]:
def weighted_mean_power(gauge_df, altimetry_df, power=5):
    
    # Baseline differences to best performing model, then calculate 
    # power weighting
    x = altimetry_df.subtract(altimetry_df.min(axis=1), axis=0)
    altimetry_weights = (1 - x.clip(0, 1)) ** power
    
    # Normalise to sum to 1
    altimetry_weights = altimetry_weights.divide(altimetry_weights.sum(axis=1), axis=0)       
    
    return (gauge_df * altimetry_weights).sum(axis=1)

def weighted_mean_power_top_n(gauge_df, altimetry_df, power=5, top_n=3):
    
    # Calculate ranks for each model
    ranks_df = altimetry_df.rank(axis=1)
    
    # Baseline differences to best performing model, then calculate 
    # power weighting
    x = altimetry_df.subtract(altimetry_df.min(axis=1), axis=0)
    altimetry_weights = (1 - x.clip(0, 1)) ** power
    altimetry_weights = altimetry_weights.where(ranks_df <= top_n, 0)
    
    # Normalise to sum to 1
    altimetry_weights = altimetry_weights.divide(altimetry_weights.sum(axis=1), axis=0)       
    
    return (gauge_df * altimetry_weights).sum(axis=1)


def mean_top_n(gauge_df, altimetry_df, top_n=5):
    
    # Calculate ranks for each model
    ranks_df = altimetry_df.rank(axis=1)
    
    # Calculate mean of top ranked models
    return gauge_df.where(ranks_df <= top_n).mean(axis=1)


def median_top_n(gauge_df, altimetry_df, top_n=5):
    
    # Calculate ranks for each model
    ranks_df = altimetry_df.rank(axis=1)
    
    # Calculate mean of top ranked models
    return gauge_df.where(ranks_df <= top_n).median(axis=1)    
    

In [ ]:
# Prepare data
test_df = tide_df.set_index("site_name", append=True).copy()
test_df["site_code"] = gauge_df.index.get_level_values("site_code").values
test_df["tide_gauge"] = gauge_df.sea_level.values

# Apply different weighting schemes
test_df["median"] = test_df[model_names].median(axis=1)
test_df["median_top5"] = median_top_n(test_df[model_names], xtrack_df, top_n=5)
test_df["median_top3"] = median_top_n(test_df[model_names], xtrack_df, top_n=3)
test_df["mean"] = test_df[model_names].mean(axis=1)
test_df["mean_top5"] = mean_top_n(test_df[model_names], xtrack_df, top_n=5)
test_df["mean_top3"] = mean_top_n(test_df[model_names], xtrack_df, top_n=3)
test_df["weighted_mean_power2"] = weighted_mean_power(
    test_df[model_names], xtrack_df, power=2
)
test_df["weighted_mean_power5"] = weighted_mean_power(
    test_df[model_names], xtrack_df, power=5
)
test_df["weighted_mean_power10"] = weighted_mean_power(
    test_df[model_names], xtrack_df, power=10
)
test_df["weighted_mean_power5_top3"] = weighted_mean_power_top_n(test_df[model_names], xtrack_df, power=5, top_n=3)
test_df["weighted_mean_power5_top5"] = weighted_mean_power_top_n(test_df[model_names], xtrack_df, power=5, top_n=5)
test_df["weighted_mean_power10_top3"] = weighted_mean_power_top_n(test_df[model_names], xtrack_df, power=10, top_n=3)
test_df["weighted_mean_power10_top5"] = weighted_mean_power_top_n(test_df[model_names], xtrack_df, power=10, top_n=5)

# Reshape to long format
tide_df_long = test_df.melt(
    ignore_index=False,
    id_vars=["tide_gauge", "site_code"],
    value_name="tide_m",
)

# Calculate statistics against tide gauge data
accuracy_df = tide_df_long.groupby(["tide_model"])[["tide_gauge", "tide_m"]].apply(
    lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m)
)
accuracy_df.sort_values("RMSE").style.background_gradient(
    cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5
)

In [ ]:
# Compare performance at worst sites
accuracy_sites_df = tide_df_long.groupby(["tide_model", "site_code"])[
    ["tide_gauge", "tide_m"]
].apply(lambda x: eval_metrics(x=x.tide_gauge, y=x.tide_m))
accuracy_sites_df.groupby(["tide_model"]).quantile(0.9).sort_values(
    "RMSE"
).style.background_gradient(cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5)


In [ ]:
accuracy_sites_df.query('site_code == "60710"').sort_values(
    "RMSE"
).style.background_gradient(cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5)

In [ ]:
accuracy_sites_df.groupby(["tide_model"]).quantile(0.95).sort_values(
    "RMSE"
).style.background_gradient(cmap="RdBu_r", subset=["RMSE"], vmin=0.15, vmax=0.5)

In [ ]:
ranked_sites_df = accuracy_sites_df.groupby("site_code")["RMSE"].transform(func='rank').subtract(0).reset_index()

sns.displot(
    ranked_sites_df, x="RMSE", col="tide_model", col_wrap=4,
    binwidth=1, height=3, facet_kws=dict(margin_titles=True),
)

In [ ]:
sns.displot(
    accuracy_sites_df.clip(0,1), x="RMSE", col="tide_model", col_wrap=4,
    binwidth=0.05, height=3, facet_kws=dict(margin_titles=True),
)

In [ ]:
# Best results overall
accuracy_sites_df.loc[accuracy_sites_df.query("tide_model != 'EOT20'").groupby("site_name").RMSE.idxmin()
].reset_index().tide_model.value_counts()
    

In [ ]:

# Step 1: Group by the first level of the multi-index ('y')
grouped = accuracy_sites_df.groupby(level=0)

# Step 2: Sort within each group by "x" in descending order
sorted_groups = grouped.apply(lambda x: x.sort_values("RMSE", ascending=False))

# sorted_groups.reset_index(inplace=True)

# Step 3: Retrieve the second highest value for each group
second_highest_per_group = sorted_groups.groupby(level=0).nth(1).reset_index()

# Display the result
print(second_highest_per_group)

In [ ]:
# Step 1: Group by the first level of the multi-index ('y')
grouped = accuracy_sites_df.groupby(level='site_name')

# Step 2: Sort within each group by "x" in descending order and get the second highest
second_highest_per_group = grouped.apply(lambda x: x.nlargest(2, 'RMSE')).groupby('site_name').nth(1).reset_index()

# Display the result
print(second_highest_per_group)

In [ ]:
# Best results overall
accuracy_sites_df.loc[
    accuracy_sites_df.sort_values("RMSE").groupby("site_name").RMSE.nth(0).index
].reset_index().tide_model.value_counts()

In [ ]:
accuracy_sites_df.sort_values("RMSE").groupby("site_name").RMSE.apply(lambda t: t.iloc[1])

In [ ]:
with pd.option_context("display.max_rows", 500):
    display(xtrack_df)

In [ ]:
with pd.option_context("display.max_rows", 500):
    display(
        xtrack_weights.style.background_gradient(
            cmap="RdBu", axis=None, vmin=0, vmax=0.2
        )
    )

In [ ]:
xtrack_weights.mean(axis=0).to_frame("Average weighting").sort_values("Average weighting", ascending=False)

### Test aggregation

In [ ]:
xtrack_df

In [ ]:
((gauge_df_long.groupby(
    "site_name"
).mean() / gauge_df_long.groupby("site_name").apply(
    lambda x: x.nsmallest(5, columns="X-TRACK altimetry (metres RMS)", keep="first")
).droplevel(level=1).groupby("site_name").mean() - 1.0)[["ABSLMP tide gauge (metres RMS)"]] * 100).style.background_gradient(cmap='RdBu', axis=None, vmin=-50, vmax=50)

In [ ]:
gauge_df_long.groupby("site_name")[["X-TRACK altimetry (metres RMS)"]].rank()

In [ ]:
gauge_df_long.groupby("site_name")[["X-TRACK altimetry (metres RMS)"]].rank()

In [ ]:
gauge_df_long.groupby("site_name")[["X-TRACK altimetry (metres RMS)"]].rank()

In [ ]:
gauge_df_long["weights"] = (gauge_df_long.groupby("site_name")[["X-TRACK altimetry (metres RMS)"]].rank(ascending=False) - 2).clip(0, 5)
gauge_df_long["weights"] = (gauge_df_long.groupby("site_name")[["X-TRACK altimetry (metres RMS)"]].rank(ascending=False))
gauge_df_long.loc["Broome"]

In [ ]:
def weighted_mean(df, values, weights, groupby):
    df = df.copy()
    grouped = df.groupby(groupby)
    df['weighted_average'] = df[values] / grouped[weights].transform('sum') * df[weights]
    return grouped['weighted_average'].sum(min_count=1) #min_count is required for Grouper objects

def weighted_median(df, val, weight):
    df_sorted = df.sort_values(val)
    cumsum = df_sorted[weight].cumsum()
    cutoff = df_sorted[weight].sum() / 2.
    return df_sorted[cumsum >= cutoff][val].iloc[0]

weighted_mean(gauge_df_long, values="ABSLMP tide gauge (metres RMS)", weights="weights", groupby="site_name") 
# gauge_df_long.groupby("site_name").apply(lambda x: weighted_median(x, val="ABSLMP tide gauge (metres RMS)", weight="weights"))


In [ ]:
gauge_df_long.groupby("site_name").mean()["ABSLMP tide gauge (metres RMS)"]

In [ ]:
weighted_df = gauge_df_long.groupby("site_name").first()
weighted_df["weights"] = gauge_df_long.groupby("site_name")[["X-TRACK altimetry (metres RMS)"]].rank()


#     [["tide_model", "X-TRACK altimetry (metres RMS)"]].rank()   #axis=0, numeric_only=True).loc["Stony Point"]

In [ ]:
df1 = pd.DataFrame(index=("a", "a", "b", "b"), data={"test": [1, 2, 3, 4]})
df1

In [ ]:
df2 = pd.DataFrame(index=("a", "b"), data={"test": [0, 2]})
df2

In [ ]:
df1 * df2

In [ ]:
gauge_df_long.groupby("site_name").mean()

### Pytides

In [ ]:
from pytides.tide import Tide

demeaned = water_level - water_level.mean()
tide = Tide.decompose(demeaned, water_level.index)
(demeaned - tide.at(water_level.index)).plot(figsize=(13, 10))